# **PYTHON CHECKER GAME PLAYED WITH SMART COMPUTER AI**
### Creator: Lam Tran *(461300)*, Tri Kieu *(464376)*

This purpose of this project/assignment is to create game-playing interface called checker (or draughts) using Pygame package that played with created with a programmed AI computer. AI computer can be made with the method of MinMax algorithms which mostly used in making board game interfaces. 

In [1]:
import pygame

# Initialize the size of Board
WIDTH, HEIGHT = 600,600
ROW, COL = 8,8

# Initialize the size of Square
SQUARE_SIZE = HEIGHT // ROW

# COLOR
BLACK = (0,0,0)
RED = (255,102,102)
WHITE = (224,224,224)

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Board:
    def __init__(self):
        self.board = []
        self.chosen_piece = 0
        self.white_left = self.red_left = 12
        self.white_king = self.red_king =0

    # First: Fill the whole board with color Black.
    # Second: Then the red square is placed. It starts from the top left (coordinate (0,0)).
    # cor_x, cor_y shows where to draw the sqaure (x and y coordinate).
    
    def draw_board(self, win):
        win.fill(BLACK)
        for row in range (ROW):
            for col in range(row % 2, ROW ,2):
                cor_x = row * SQUARE_SIZE
                cor_y = col * SQUARE_SIZE
                pygame.draw.rect(win,RED, (cor_x,cor_y,SQUARE_SIZE,SQUARE_SIZE))
    
    


In [3]:
class main:
    status = True
    board = Board()
    FPS = 60

    WINDOW = pygame.display.set_mode((WIDTH,HEIGHT))
    pygame.display.set_caption("Checkers")

    clock = pygame.time.Clock()

    while status == True:
        clock.tick(FPS)

        for action in pygame.event.get():
            if action == pygame.QUIT:
                status = False
            
            if action == pygame.MOUSEBUTTONDOWN:
                pass
            
        board.draw_board(WINDOW)
        pygame.display.update()
    
    pygame.quit()



KeyboardInterrupt: 